In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.ticker as ticker
import matplotlib.pylab as pl
from itertools import cycle
import matplotlib.gridspec as gridspec
import glob
import collections
import math
import re
import os
from collections import defaultdict

This notebook is used to perform the shannon index etc calculations for the multispecies project.

It's based on the code used for the shannon index etc calculations for the biofilm threshold theory species composition calculations

In [3]:
nRuns = 100
duration = 4368 #duration of sims in hours - equivalent to 26 weeks
dates = ["24-Sep-2020"] #dates the simulations were performed on
pc_res = [14, 15] #percentages of the populations which are resistant to the applied biocide
phase2_str = "phase2"

#parameters for the log normal distributions used
#[scale, sigma]
log_norm_params_14pcRes = [2.703747953786337, 0.5690825284230452]
log_norm_params_15pcRes = [2.6133256846855746, 0.6260058161550592]
log_norm_params_16pcRes = [2.47772924764521, 0.7060073500033884]

In [4]:
def getFilepathToGenoRuns(date, pc_res, phase):
    '''
    creates a string with the file location of the genotype distributions (all the run_ID files)
    '''
    
    return "geno_distb_data_"+phase+"/"+str(pc_res)+"_resistant-"+date+"/"

def getEventCountersDataframe(date, pc_res, phase, sigma, duration):
    
    return pd.read_csv("geno_distb_data_"+phase+"/"+str(pc_res)+"_resistant-"+date+"/"+str(pc_res)+"_resistant-"+date+"-event_counters-sigma="+"{:.5f}".format(sigma)+"-t="+str(duration)+".0.csv")


def getListOfMeasurementTimes(directory_name):
    '''
    for each runID directory, this gets the filenames and extracts a list of the times they were sampled at.
    directory_name is of form path_to_files/runID_<n>
    
    returns: sorted list of the time vals, in string form with 2 decimal places
    '''
    time_list = []
    def get_numbers_from_filename(filename):
        return re.search(r'(\d+(?:\.\d+)?)', filename).group(0)
    
    for filename in os.listdir(directory_name):
        time_list.append(float(get_numbers_from_filename(filename)))

    return ["{:.2f}".format(float(t)) for t in sorted(time_list)]

In [5]:
def shannonIndexAndEquitabilitySolo(geno_dict):
    '''
    For a single run, this calculates the shannon variables H, E, S.
    Outputs a .csv file with the calculated variables over time.
    Can then combine these into a dictionary of dataframes later.
    '''
    
    times = []
    nBac_t = defaultdict(list) #no. of bacteria over time
    H_t = defaultdict(list) #shannon index over time
    E_t = defaultdict(list) #shannon equitability over time
    S_t = defaultdict(list) #no. of species over time
    
    times = geno_dict.keys()
    #print(times)
    for time_key in times:

        #here we create an array with all the genotypes in it and remove any nans
        geno_vals = geno_dict[time_key].values.flatten()[~np.isnan(geno_dict[time_key].values.flatten())]
        nTot = geno_vals.size #total number of bacteria in the population
        genoCounts = collections.Counter(geno_vals) #number of members of each bacterial species in the system

        H = sum([-n/nTot*math.log(n/nTot) for _, n in genoCounts.items()]) #shannon index of this run at time t
        S = len(genoCounts.keys()) #no. of different species in the system
        logS_adjusted = 1 if S == 1 else math.log(S)
        E = H/logS_adjusted #shannon equitability

        nBac_t[time_key].append(int(nTot))
        H_t[time_key].append(H)
        E_t[time_key].append(E)
        S_t[time_key].append(S)
        
    #this is a very poor way of doing things, but in a rush and just trying to 
    #get a good enough job done atm
    nBac_t_list = [b[0] for b in nBac_t.values()]
    H_t_list = [h[0] for h in H_t.values()]
    E_t_list = [e[0] for e in E_t.values()]
    S_t_list = [s[0] for s in S_t.values()]
    
    return list(H_t.keys()), nBac_t_list, H_t_list, E_t_list, S_t_list

In [6]:
def shannonIndexAndEquitabilitySolo_EDGE(geno_dict):
    '''
    This does the same as the other shannon processing stuff, but just for a single run.
    We'll save all the individual calculations to .csv files, then combine them into a dataframe later
    
    This is just for the edge microhabitats
    '''
    
    times = []
    nBac_t = defaultdict(list) #no. of bacteria over time
    H_t = defaultdict(list) #shannon index over time
    E_t = defaultdict(list) #shannon equitability over time
    S_t = defaultdict(list) #no. of species over time
    
    times = geno_dict.keys()
    #print(times)
    for time_key in times:
        
        #get the last key in this timestep, hopefully it's the edge one
        edge_mh_key = geno_dict[time_key].keys()[-1]
        #here we create an array with all the genotypes in it and remove any nans (this version should just be of the edge values)
        geno_vals = geno_dict[time_key][edge_mh_key].values.flatten()[~np.isnan(geno_dict[time_key][edge_mh_key].values.flatten())]
        nTot = geno_vals.size #total number of bacteria in the population
        genoCounts = collections.Counter(geno_vals) #number of members of each bacterial species in the system

        H = sum([-n/nTot*math.log(n/nTot) for _, n in genoCounts.items()]) #shannon index of this run at time t
        S = len(genoCounts.keys()) #no. of different species in the system
        logS_adjusted = 1 if S <= 1 else math.log(S)
        E = H/logS_adjusted #shannon equitability

        nBac_t[time_key].append(int(nTot))
        H_t[time_key].append(H)
        E_t[time_key].append(E)
        S_t[time_key].append(S)
        
    #this is a very poor way of doing things, but in a rush and just trying to 
    #get a good enough job done atm
    nBac_t_list = [b[0] for b in nBac_t.values()]
    H_t_list = [h[0] for h in H_t.values()]
    E_t_list = [e[0] for e in E_t.values()]
    S_t_list = [s[0] for s in S_t.values()]
    
    return list(H_t.keys()), nBac_t_list, H_t_list, E_t_list, S_t_list

In [7]:
def writeShannonCalculationsToFile(t_data, nBac_data, H_data, E_data, S_data, pc_res, date, phase_val, growth_val, filename):
    '''
    it takes an absolute age to load in all the genotype data, so this method will write the calculated values to a file
    '''
    #create a dataframe containing all the calculated values
    #first we need a dictionary with the data in it
    collated_data = {'t':t_data, 'nBac':nBac_data, 'H':H_data, 'E':E_data, 'S':S_data}
    df = pd.DataFrame(collated_data)
    df.to_csv("shannon_calculations_"+phase_val+"_"+growth_val+"/"+str(pc_res)+"_pc_res-"+date+"/"+filename)

In [8]:
def writeShannonCalculationsToFile_EDGE(t_data, nBac_data, H_data, E_data, S_data, pc_res, date, phase_val, growth_val, filename):
    '''
    it takes an absolute age to load in all the genotype data, so this method will write the calculated values to a file
    this method is just for the edge data
    '''
    #create a dataframe containing all the calculated values
    #first we need a dictionary with the data in it
    collated_data = {'t':t_data, 'nBac':nBac_data, 'H':H_data, 'E':E_data, 'S':S_data}
    df = pd.DataFrame(collated_data)
    df.to_csv("shannon_calculations_"+phase_val+"_"+growth_val+"_EDGE/"+str(pc_res)+"_pc_res-"+date+"_"+str(growth_val)+"_EDGE/"+filename)

In [9]:
def readGenoDistbAndProcessShannonData(directoryPath, pc_res, date, phase_val, growth_val, runID):
    '''
    This loads in all the genotype data for a single run.
    
    growth_val can either be "GROWTH" for runs which exhibit growth, "NOGROWTH" for runs which remain in the first microhabitat,
    or "" if you want to 
    '''
    runID_key = "runID_"+str(runID)
    print(runID_key)
    filepath_runID = directoryPath+"/"+runID_key
    geno_time_dict = {} #dictionary containing geno dataframes for each timestep

    time_list = getListOfMeasurementTimes(filepath_runID) #sorted list of the times that the genos were sampled at in this run
    
    for t in time_list:
            
        filepath_time = filepath_runID+"/geno_distb-t="+t+".csv"

        #need to swap the rows and columns so that the microhabitat is the key in the dataframe
        #geno_df = pd.read_csv(filename, header=None).T
        geno_df = pd.DataFrame([line.strip().split(',') for line in open(filepath_time, 'r')]).T
        #geno
        new_header = geno_df.iloc[0] #grab the first row for the header
        geno_df = geno_df[1:] #take the data less the header row
        geno_df.columns = new_header #set the header row as the df header

        geno_df = geno_df.astype(float)

        #round the time to the nearest integer value to make reading it in easier
        #the [-3] is so the decimal point and decimal numbers are removed when casting the string to an int
        geno_time_dict[int(t[:-3])] = geno_df

    #we now have the geno distb loaded, so can process it
    t_list, nBac_list, H_vs_t_list, E_vs_t_list, S_vs_t_list = shannonIndexAndEquitabilitySolo(geno_time_dict)
    #write the data to file 
    writeShannonCalculationsToFile(t_list, nBac_list, H_vs_t_list, E_vs_t_list, S_vs_t_list, pc_res=pc_res, date=date, phase_val=phase_val, growth_val=growth_val,
                                   filename="shannon_calculations-"+str(pc_res)+"_pc_res-runID_"+str(runID)+".csv")
    del(geno_time_dict)

In [16]:
def readGenoDistbAndProcessShannonData_EDGE(directoryPath, pc_res, date, phase_val, growth_val, runID):
    '''
    Loading all the dataframes into one master dictionary was causing serious memory issues.
    So here we'll just load in the geno data and process it for a single run at a time.
    this method is just for the edge microhabitats
    '''
    runID_key = "runID_"+str(runID)
    print(runID_key)
    filepath_runID = directoryPath+"/"+runID_key
    geno_time_dict = {} #dictionary containing geno dataframes for each timestep

    time_list = getListOfMeasurementTimes(filepath_runID) #sorted list of the times that the genos were sampled at in this run
    
    for t in time_list:
            
        filepath_time = filepath_runID+"/geno_distb-t="+t+".csv"

        #need to swap the rows and columns so that the microhabitat is the key in the dataframe
        #geno_df = pd.read_csv(filename, header=None).T
        geno_df = pd.DataFrame([line.strip().split(',') for line in open(filepath_time, 'r')]).T
        #geno
        new_header = geno_df.iloc[0] #grab the first row for the header
        geno_df = geno_df[1:] #take the data less the header row
        geno_df.columns = new_header #set the header row as the df header

        geno_df = geno_df.astype(float)

        #round the time to the nearest integer value to make reading it in easier
        #the [-3] is so the decimal point and decimal numbers are removed when casting the string to an int
        geno_time_dict[int(t[:-3])] = geno_df

    #we now have the geno distb loaded, so can process it
    t_list, nBac_list, H_vs_t_list, E_vs_t_list, S_vs_t_list = shannonIndexAndEquitabilitySolo_EDGE(geno_time_dict)
    #write the data to file 
    writeShannonCalculationsToFile_EDGE(t_list, nBac_list, H_vs_t_list, E_vs_t_list, S_vs_t_list, pc_res=pc_res, date=date, phase_val=phase_val, growth_val=growth_val,
                                   filename="shannon_calculations-"+str(pc_res)+"_pc_res-runID_"+str(runID)+".csv")
    del(geno_time_dict)

In [17]:
pc_res_14_24_Sep_filepath = getFilepathToGenoRuns(date=dates[0], pc_res=pc_res[0], phase=phase2_str)
pc_res_15_24_Sep_filepath = getFilepathToGenoRuns(date=dates[0], pc_res=pc_res[1], phase=phase2_str)
print(pc_res_14_24_Sep_filepath)

geno_distb_data_phase2/14_resistant-24-Sep-2020/


The discrepancy in species composition for runs which exhibit growth, and runs which don't, I believe is throwing off the shannon calculations somewhat.

Therefore, using the event_counters dataframe, we can seperate the runs into categories of GROWTH (thickess > 0) and NO_GROWTH.

In [12]:
event_counters_14pc_24Sep = getEventCountersDataframe(dates[0], pc_res[0], phase2_str, log_norm_params_14pcRes[1], duration)
event_counters_15pc_24Sep = getEventCountersDataframe(dates[0], pc_res[1], phase2_str, log_norm_params_15pcRes[1], duration)

In [14]:
GROWTH_RUNS_14pc_24Sep = list(event_counters_14pc_24Sep["runID"][(event_counters_14pc_24Sep["bf_thickness"] > 0)])
GROWTH_RUNS_15pc_24Sep = list(event_counters_15pc_24Sep["runID"][(event_counters_15pc_24Sep["bf_thickness"] > 0)])

NOGROWTH_RUNS_14pc_24Sep = list(event_counters_14pc_24Sep["runID"][~(event_counters_14pc_24Sep["bf_thickness"] > 0)])
NOGROWTH_RUNS_15pc_24Sep = list(event_counters_15pc_24Sep["runID"][~(event_counters_15pc_24Sep["bf_thickness"] > 0)])

In [20]:
for runID in GROWTH_RUNS_14pc_24Sep:
    readGenoDistbAndProcessShannonData_EDGE(directoryPath=pc_res_14_24_Sep_filepath, pc_res=pc_res[0], date=dates[0], phase_val=phase2_str, growth_val="GROWTH", runID=runID)

runID_17
runID_38
runID_48
runID_49
runID_66
runID_79


In [19]:
for runID in GROWTH_RUNS_15pc_24Sep:
    readGenoDistbAndProcessShannonData_EDGE(directoryPath=pc_res_15_24_Sep_filepath, pc_res=pc_res[1], date=dates[0], phase_val=phase2_str, growth_val="GROWTH", runID=runID)

runID_37
runID_47
runID_49
runID_56
runID_57
runID_62
runID_70
runID_77
runID_83
runID_84
runID_89
runID_91
runID_94
